In [2]:
import os
import pickle
import pandas as pd
import numpy as np

import networkx as nx

import torch
import torch.nn.functional as F
from torch import nn
from torch.utils.data import Dataset, DataLoader, random_split

from torch_geometric.data import Data, DataLoader as PyGDataLoader
from torch_geometric.utils import from_networkx
from torch_geometric.nn import GCNConv, GINEConv, global_mean_pool, BatchNorm
from torch_geometric.nn.conv.gcn_conv import gcn_norm

from sklearn.metrics import r2_score, mean_squared_error



In [3]:
import warnings
warnings.filterwarnings('ignore')


In [4]:
class MolDataset(Dataset):
    """
    A custom dataset that:
      - Reads external factors from CSV
      - Loads the corresponding pickle for the molecule's graph
      - Converts it into a PyG Data object
    """
    def __init__(self,
                 raw_dataframe: pd.DataFrame,
                 nx_graph_dict: dict,
                 *,
                 component_col: str,
                 global_state_cols: list[str],
                 label_col: str,
                 transform=None):
        """
        Args:

        """
        self.raw_dataframe = raw_dataframe
        self.nx_graph_dict = nx_graph_dict
        self.component_col = [component_col] if type(component_col) is str else component_col
        self.global_state_cols = global_state_cols
        self.label_col = [label_col] if type(label_col) is str else label_col
        self.transform = transform
        
        required_cols = set(self.global_state_cols + self.label_col + self.component_col)
        for col in required_cols:
            if col not in self.raw_dataframe.columns:
                raise ValueError(f"Missing column in DataFrame: '{col}'")


    def __len__(self):
        return len(self.raw_dataframe)

    def __getitem__(self, idx):
        row = self.raw_dataframe.iloc[idx]
        
        # 1. Load the molecule graph
        component_name = row[self.component_col[0]]  # e.g. "C23"
        pyg_data = self.nx_graph_dict[component_name]

        # 3. Prepare the external factors
        #    Convert selected columns into a float tensor
        externals = torch.tensor(row[self.global_state_cols].values.astype(float), dtype=torch.float)
        externals = externals.unsqueeze(0)

        # 4. Prepare the label (regression target)
        label = torch.tensor([row[self.label_col][0]], dtype=torch.float)

        # 5. Attach externals & label to the Data object for use in the model
        #    (We can store them in Data object attributes if you like)
        pyg_data.externals = externals  # 1D vector of external factors
        pyg_data.y = label  # shape [1]

        if self.transform:
            pyg_data = self.transform(pyg_data)

        return pyg_data


In [5]:

def networkx_to_pyg(nx_graph):
    """
    Convert a networkx graph to a torch_geometric.data.Data object.
    This is a basic template; adjust for your actual node/edge features.
    """
    # Sort nodes to ensure consistent ordering
    # e.g. node 0, node 1, ...
    # In some networkx graphs, node labels might be strings. We’ll map them to integers.
    node_mapping = {node: i for i, node in enumerate(nx_graph.nodes())}

    # Build lists for PyG
    x_list = []
    edge_index_list = []
    edge_attr_list = []

    for node in nx_graph.nodes(data=True):
        original_id = node[0]
        attrs = node[1]
        # Example: 'symbol' might be in attrs, etc.
        # For demonstration, let's store only "symbol" as a simple categorical embedding
        # You might do something more sophisticated (e.g., one-hot) for real usage
        symbol = attrs.get("symbol", "C")
        # Convert symbol to a simple ID (C=0, H=1, etc.) or some vector
        # We'll do a naive approach here:
        symbol_id = 0 if symbol == "C" else 1 if symbol == "H" else 2
        
        x_list.append([symbol_id])

    for u, v, edge_attrs in nx_graph.edges(data=True):
        u_idx = node_mapping[u]
        v_idx = node_mapping[v]
        edge_index_list.append((u_idx, v_idx))
        # Possibly store bond features: "bond_index", "bde_pred", etc.
        bde_pred = edge_attrs.get("bde_pred", 0.0)
        if bde_pred is None:
            bde_pred = 0.0
        bdfe_pred = edge_attrs.get("bdfe_pred", 0.0)
        if bdfe_pred is None:
            bdfe_pred = 0.0
        edge_attr_list.append([bde_pred, bdfe_pred])
    
    # Convert to torch tensors
    x = torch.tensor(x_list, dtype=torch.float)  # shape [num_nodes, num_node_features]
    edge_index = torch.tensor(edge_index_list, dtype=torch.long).t().contiguous()  # shape [2, num_edges]
    edge_attr = torch.tensor(edge_attr_list, dtype=torch.float)  # shape [num_edges, edge_feat_dim]

    data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr)
    return data


In [6]:
class GINE_Regression(nn.Module):
    def __init__(self,
                 node_in_dim: int,
                 edge_in_dim: int,
                 external_in_dim: int,
                 hidden_dim: int = 128,
                 num_layers: int = 3,
                 dropout: float = 0.1):
        """
        A more 'realistic' GNN for regression, using GINEConv layers + edge attributes.
        
        Args:
            node_in_dim (int): Dim of node features (e.g. 1 or 3).
            edge_in_dim (int): Dim of edge features (e.g. 2 for [bde_pred, bdfe_pred]).
            external_in_dim (int): Dim of external factor features (e.g. 6).
            hidden_dim (int): Hidden embedding size for GNN layers.
            num_layers (int): Number of GNN layers.
            dropout (float): Dropout probability.
        """
        super().__init__()
        
        # A learnable linear transform for edge features (required by GINEConv's "nn" argument):
        # Typically GINEConv uses a small MLP to incorporate edge_attr into the message.
        self.edge_encoder = nn.Sequential(
            nn.Linear(edge_in_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim)
        )
        
        # A learnable linear transform for node features:
        self.node_encoder = nn.Linear(node_in_dim, hidden_dim)
        
        # Create multiple GINEConv layers
        self.convs = nn.ModuleList()
        self.bns = nn.ModuleList()
        
        for _ in range(num_layers):
            # GINEConv requires an MLP for node update:
            # We'll use a simple 2-layer MLP
            net = nn.Sequential(
                nn.Linear(hidden_dim, hidden_dim),
                nn.ReLU(),
                nn.Linear(hidden_dim, hidden_dim)
            )
            conv = GINEConv(nn=net)
            self.convs.append(conv)
            self.bns.append(BatchNorm(hidden_dim))  # batch norm for stability

        self.dropout = nn.Dropout(p=dropout)

        # An MLP to process external factors
        self.externals_mlp = nn.Sequential(
            nn.Linear(external_in_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(p=dropout),
            nn.Linear(hidden_dim, hidden_dim)
        )

        # Final regression MLP after pooling + external embedding
        self.final_regressor = nn.Sequential(
            nn.Linear(hidden_dim + hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(p=dropout),
            nn.Linear(hidden_dim, 1)
        )

    def forward(self, data):
        """
        Args:
            data: PyG Data object, expected fields:
                - x: Node features [num_nodes, node_in_dim]
                - edge_index: [2, num_edges]
                - edge_attr: [num_edges, edge_in_dim]
                - batch: [num_nodes] mapping each node to a graph ID
                - externals: [batch_size, external_in_dim]
        Returns:
            A tensor of shape [batch_size], the predicted regression value.
        """
        x, edge_index, edge_attr, batch = data.x, data.edge_index, data.edge_attr, data.batch
        
        # 1) Encode node features and edge features
        x = self.node_encoder(x)                 # [num_nodes, hidden_dim]
        edge_emb = self.edge_encoder(edge_attr)  # [num_edges, hidden_dim]
        
        # 2) Pass through multiple GINEConv layers
        for conv, bn in zip(self.convs, self.bns):
            x = conv(x, edge_index, edge_emb)
            x = bn(x)
            x = F.relu(x)
            x = self.dropout(x)

        # 3) Global pooling to get graph embedding
        graph_emb = global_mean_pool(x, batch)  # [batch_size, hidden_dim]

        # 4) Process external factors
        ext_emb = self.externals_mlp(data.externals)  # [batch_size, hidden_dim]

        # 5) Combine + final regression
        combined = torch.cat([graph_emb, ext_emb], dim=-1)  # [batch_size, hidden_dim * 2]
        out = self.final_regressor(combined).squeeze(-1)    # [batch_size]
        return out


In [7]:
def train_one_epoch(model, loader, optimizer, criterion, device):
    model.train()
    total_loss = 0.0
    count = 0
    for batch_data in loader:
        batch_data = batch_data.to(device)
        optimizer.zero_grad()
        preds = model(batch_data)               # [batch_size]
        y = batch_data.y.to(device).view(-1)    # [batch_size]
        loss = criterion(preds, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * batch_data.num_graphs
        count += batch_data.num_graphs
    return total_loss / count if count > 0 else 0.0

def validate(model, loader, criterion, device):
    model.eval()
    total_loss = 0.0
    count = 0
    with torch.no_grad():
        for batch_data in loader:
            batch_data = batch_data.to(device)
            preds = model(batch_data)
            y = batch_data.y.to(device).view(-1)
            loss = criterion(preds, y)
            total_loss += loss.item() * batch_data.num_graphs
            count += batch_data.num_graphs
    return total_loss / count if count > 0 else 0.0


In [8]:


def evaluate_model(model, loader, device):
    """
    Evaluate the model on a dataset loader and compute R² and RMSE.

    Args:
        model (nn.Module): The trained GNN model.
        loader (DataLoader): The PyG DataLoader for the evaluation dataset.
        device (torch.device): The device to run on.
    
    Returns:
        r2 (float): Coefficient of determination.
        rmse (float): Root Mean Squared Error.
    """
    model.eval()
    y_true, y_pred = [], []

    with torch.no_grad():
        for batch in loader:
            batch = batch.to(device)
            preds = model(batch)
            y_true.append(batch.y.cpu())
            y_pred.append(preds.cpu())

    # If your labels are stored as tensors with an extra dimension, use .squeeze() if needed.
    y_true = torch.cat(y_true).numpy().squeeze()
    y_pred = torch.cat(y_pred).numpy().squeeze()

    r2 = r2_score(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))

    print(f"R²: {r2:.4f}")
    print(f"RMSE: {rmse:.4f}")

    return r2, rmse


In [9]:
env_file = r"C:\Users\80710\OneDrive - Imperial College London\2025 engineering\GNN molecules\graph_pickles\dataset02.xlsx"

data = pd.read_excel(env_file, engine='openpyxl').dropna(subset=['degradation_rate'])
data['seawater'] = data['seawater'].map({'art': 1, 'sea': 0})

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1023 entries, 0 to 1039
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   data number       1023 non-null   float64
 1   temperature       1023 non-null   float64
 2   seawater          1023 non-null   int64  
 3   concentration     1023 non-null   int64  
 4   time              1023 non-null   int64  
 5   component         1023 non-null   object 
 6   BDE               1023 non-null   float64
 7   BDFE              1023 non-null   float64
 8   energy            1023 non-null   float64
 9   degradation_rate  1023 non-null   float64
dtypes: float64(6), int64(3), object(1)
memory usage: 87.9+ KB


In [11]:
folder_path = r"C:\Users\80710\OneDrive - Imperial College London\2025 engineering\GNN molecules\graph_pickles\molecules"
graph_pickles = [f for f in os.listdir(folder_path) if f.endswith(".pkl")]

In [12]:
import os

base_dir = r"C:\Users\80710\OneDrive - Imperial College London\2025 engineering\GNN molecules\graph_pickles\molecules"

if os.path.exists(base_dir):
    print("Directory exists:", base_dir)
    print("Files in directory:", os.listdir(base_dir))
else:
    print(f"Error: Directory {base_dir} does not exist!")

Directory exists: C:\Users\80710\OneDrive - Imperial College London\2025 engineering\GNN molecules\graph_pickles\molecules
Files in directory: ['gpickle_graph_0.pkl', 'gpickle_graph_1.pkl', 'gpickle_graph_10.pkl', 'gpickle_graph_11.pkl', 'gpickle_graph_12.pkl', 'gpickle_graph_13.pkl', 'gpickle_graph_14.pkl', 'gpickle_graph_15.pkl', 'gpickle_graph_16.pkl', 'gpickle_graph_17.pkl', 'gpickle_graph_18.pkl', 'gpickle_graph_19.pkl', 'gpickle_graph_2.pkl', 'gpickle_graph_3.pkl', 'gpickle_graph_4.pkl', 'gpickle_graph_5.pkl', 'gpickle_graph_6.pkl', 'gpickle_graph_7.pkl', 'gpickle_graph_8.pkl', 'gpickle_graph_9.pkl']


In [13]:
compounds = data.component.unique()
graphs_dict = {}

for compound, graph_pickle in zip(compounds, graph_pickles):
    with open(os.path.join(base_dir, graph_pickle), 'rb') as f:
        graph = pickle.load(f)
        graphs_dict[compound] = networkx_to_pyg(graph)


In [14]:

dataset = MolDataset(
    raw_dataframe=data,
    nx_graph_dict=graphs_dict,
    component_col="component",
    global_state_cols=["temperature", "concentration", "time", "seawater"],
    label_col="degradation_rate",
    transform=None
)

# Simple random split for demonstration
train_size = int(0.8 * len(dataset))  # 80% train
val_size   = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = PyGDataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader   = PyGDataLoader(val_dataset, batch_size=16, shuffle=False)



In [15]:

# -----------------------------------
# 2) Instantiate model + optimizer
# -----------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [16]:
model = GINE_Regression(
    node_in_dim=1,
    edge_in_dim=2,
    external_in_dim=4,
    hidden_dim=16,
    num_layers=5,
    dropout=0.1
).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
criterion = torch.nn.MSELoss()


In [17]:

# -----------------------------------
# 3) Training Loop
# -----------------------------------
num_epochs = 500
for epoch in range(1, num_epochs+1):
    train_loss = train_one_epoch(model, train_loader, optimizer, criterion, device)
    val_loss = validate(model, val_loader, criterion, device)

    if epoch % 10 == 0:
        print(f"[Epoch {epoch}] train_loss: {train_loss:.4f}, val_loss: {val_loss:.4f}")

# Optionally, test or save the model
# torch.save(model.state_dict(), "trained_gine_model.pt")

[Epoch 10] train_loss: 0.1527, val_loss: 0.0777
[Epoch 20] train_loss: 0.1020, val_loss: 0.0644
[Epoch 30] train_loss: 0.0765, val_loss: 0.0652
[Epoch 40] train_loss: 0.0765, val_loss: 0.0595
[Epoch 50] train_loss: 0.0719, val_loss: 0.0607
[Epoch 60] train_loss: 0.0602, val_loss: 0.0608
[Epoch 70] train_loss: 0.0808, val_loss: 0.0572
[Epoch 80] train_loss: 0.0644, val_loss: 0.0691
[Epoch 90] train_loss: 0.0675, val_loss: 0.0560
[Epoch 100] train_loss: 0.0748, val_loss: 0.0604
[Epoch 110] train_loss: 0.0680, val_loss: 0.0581
[Epoch 120] train_loss: 0.0687, val_loss: 0.0534
[Epoch 130] train_loss: 0.0633, val_loss: 0.0565
[Epoch 140] train_loss: 0.0605, val_loss: 0.0618
[Epoch 150] train_loss: 0.0615, val_loss: 0.0507
[Epoch 160] train_loss: 0.0590, val_loss: 0.0493
[Epoch 170] train_loss: 0.0603, val_loss: 0.0511
[Epoch 180] train_loss: 0.0677, val_loss: 0.0526
[Epoch 190] train_loss: 0.0569, val_loss: 0.0484
[Epoch 200] train_loss: 0.0506, val_loss: 0.0485
[Epoch 210] train_loss: 0.059

In [19]:

# Example usage after training:
r2, rmse = evaluate_model(model, val_loader, device)

R²: 0.3789
RMSE: 0.2076


### OPTUNA

In [1]:
import os
import pickle
import pandas as pd
import numpy as np

import networkx as nx

import torch
import torch.nn.functional as F
from torch import nn
from torch.utils.data import Dataset, DataLoader, random_split
from torch_geometric.data import Data, DataLoader as PyGDataLoader
from torch_geometric.utils import from_networkx
from torch_geometric.nn import GINEConv, global_mean_pool, BatchNorm
from torch_geometric.nn.conv.gcn_conv import gcn_norm

from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import KFold

import warnings
warnings.filterwarnings('ignore')

# =============================================================================
#                             DATASET & PREP
# =============================================================================

class MolDataset(Dataset):
    """
    A custom dataset that:
      - Reads external factors from CSV
      - Loads the corresponding pickle for the molecule's graph
      - Converts it into a PyG Data object
    """
    def __init__(self,
                 raw_dataframe: pd.DataFrame,
                 nx_graph_dict: dict,
                 *,
                 component_col: str,
                 global_state_cols: list[str],
                 label_col: str,
                 transform=None):
        """
        Args:
            raw_dataframe: The input dataframe containing molecule info.
            nx_graph_dict: Dictionary mapping component names to networkx graphs.
            component_col: Column name for the component.
            global_state_cols: List of columns representing external factors.
            label_col: Column name for the regression target.
            transform: Any transform to apply to each PyG Data object.
        """
        self.raw_dataframe = raw_dataframe
        self.nx_graph_dict = nx_graph_dict
        self.component_col = [component_col] if type(component_col) is str else component_col
        self.global_state_cols = global_state_cols
        self.label_col = [label_col] if type(label_col) is str else label_col
        self.transform = transform
        
        required_cols = set(self.global_state_cols + self.label_col + self.component_col)
        for col in required_cols:
            if col not in self.raw_dataframe.columns:
                raise ValueError(f"Missing column in DataFrame: '{col}'")

    def __len__(self):
        return len(self.raw_dataframe)

    def __getitem__(self, idx):
        row = self.raw_dataframe.iloc[idx]
        
        # 1. Load the molecule graph
        component_name = row[self.component_col[0]]  # e.g. "C23"
        pyg_data = self.nx_graph_dict[component_name]

        # 2. Prepare the external factors
        externals = torch.tensor(row[self.global_state_cols].values.astype(float), dtype=torch.float)
        externals = externals.unsqueeze(0)

        # 3. Prepare the label (regression target)
        label = torch.tensor([row[self.label_col][0]], dtype=torch.float)

        # 4. Attach externals & label to the Data object
        pyg_data.externals = externals  # shape [1, external_in_dim]
        pyg_data.y = label  # shape [1]

        if self.transform:
            pyg_data = self.transform(pyg_data)

        return pyg_data


def networkx_to_pyg(nx_graph):
    """
    Convert a networkx graph to a torch_geometric.data.Data object.
    This is a basic template; adjust for your actual node/edge features.
    """
    # Sort nodes to ensure consistent ordering
    node_mapping = {node: i for i, node in enumerate(nx_graph.nodes())}

    x_list = []
    edge_index_list = []
    edge_attr_list = []

    # Node features
    for node in nx_graph.nodes(data=True):
        original_id = node[0]
        attrs = node[1]
        symbol = attrs.get("symbol", "C")
        symbol_id = 0 if symbol == "C" else 1 if symbol == "H" else 2
        x_list.append([symbol_id])

    # Edge features
    for u, v, edge_attrs in nx_graph.edges(data=True):
        u_idx = node_mapping[u]
        v_idx = node_mapping[v]
        edge_index_list.append((u_idx, v_idx))
        bde_pred = edge_attrs.get("bde_pred", 0.0) or 0.0
        bdfe_pred = edge_attrs.get("bdfe_pred", 0.0) or 0.0
        edge_attr_list.append([bde_pred, bdfe_pred])

    x = torch.tensor(x_list, dtype=torch.float)
    edge_index = torch.tensor(edge_index_list, dtype=torch.long).t().contiguous()
    edge_attr = torch.tensor(edge_attr_list, dtype=torch.float)

    data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr)
    return data


# =============================================================================
#                     BASE GNN MODEL (WITH DIM MATCH)
# =============================================================================

class GINE_Regression(nn.Module):
    """
    A GNN for regression using GINEConv layers + edge attributes,
    where all layers have the same hidden_dim (no dimension mismatch).
    """
    def __init__(self,
                 node_in_dim: int,
                 edge_in_dim: int,
                 external_in_dim: int,
                 hidden_dim: int = 128,
                 num_layers: int = 3,
                 dropout: float = 0.1):
        super().__init__()
        
        # Encode edges from edge_in_dim to hidden_dim
        self.edge_encoder = nn.Sequential(
            nn.Linear(edge_in_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim)
        )
        
        # Encode nodes from node_in_dim to hidden_dim
        self.node_encoder = nn.Linear(node_in_dim, hidden_dim)
        
        # Multiple GINEConv layers & corresponding BatchNorm
        self.convs = nn.ModuleList()
        self.bns = nn.ModuleList()
        
        for _ in range(num_layers):
            net = nn.Sequential(
                nn.Linear(hidden_dim, hidden_dim),
                nn.ReLU(),
                nn.Linear(hidden_dim, hidden_dim)
            )
            conv = GINEConv(nn=net)
            self.convs.append(conv)
            self.bns.append(BatchNorm(hidden_dim))

        self.dropout = nn.Dropout(p=dropout)

        # Process external factors
        self.externals_mlp = nn.Sequential(
            nn.Linear(external_in_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(p=dropout),
            nn.Linear(hidden_dim, hidden_dim)
        )

        # Final regression
        self.final_regressor = nn.Sequential(
            nn.Linear(hidden_dim + hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(p=dropout),
            nn.Linear(hidden_dim, 1)
        )

    def forward(self, data):
        x, edge_index, edge_attr, batch = data.x, data.edge_index, data.edge_attr, data.batch

        # Encode
        x = self.node_encoder(x)
        edge_emb = self.edge_encoder(edge_attr)
        
        # Pass through GINEConv layers
        for conv, bn in zip(self.convs, self.bns):
            x = conv(x, edge_index, edge_emb)
            x = bn(x)
            x = F.relu(x)
            x = self.dropout(x)

        # Global pooling
        graph_emb = global_mean_pool(x, batch)

        # Process external factors
        ext_emb = self.externals_mlp(data.externals)

        # Combine & regress
        combined = torch.cat([graph_emb, ext_emb], dim=-1)
        out = self.final_regressor(combined).squeeze(-1)
        return out


# =============================================================================
#                   TRAIN/VALID/EVALUATION UTILS
# =============================================================================

def train_one_epoch(model, loader, optimizer, criterion, device):
    model.train()
    total_loss = 0.0
    count = 0
    for batch_data in loader:
        batch_data = batch_data.to(device)
        optimizer.zero_grad()
        preds = model(batch_data)
        y = batch_data.y.to(device).view(-1)
        loss = criterion(preds, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * batch_data.num_graphs
        count += batch_data.num_graphs
    return total_loss / count if count > 0 else 0.0


def validate(model, loader, criterion, device):
    model.eval()
    total_loss = 0.0
    count = 0
    with torch.no_grad():
        for batch_data in loader:
            batch_data = batch_data.to(device)
            preds = model(batch_data)
            y = batch_data.y.to(device).view(-1)
            loss = criterion(preds, y)
            total_loss += loss.item() * batch_data.num_graphs
            count += batch_data.num_graphs
    return total_loss / count if count > 0 else 0.0


def evaluate_model(model, loader, device):
    """
    Evaluate the model on a dataset loader and compute R² and RMSE.
    """
    model.eval()
    y_true, y_pred = [], []

    with torch.no_grad():
        for batch in loader:
            batch = batch.to(device)
            preds = model(batch)
            y_true.append(batch.y.cpu())
            y_pred.append(preds.cpu())

    y_true = torch.cat(y_true).numpy().squeeze()
    y_pred = torch.cat(y_pred).numpy().squeeze()

    r2 = r2_score(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))

    print(f"R²: {r2:.4f}")
    print(f"RMSE: {rmse:.4f}")

    return r2, rmse


# =============================================================================
#                             DATA PREPARATION
# =============================================================================

env_file = r"C:\Users\80710\OneDrive - Imperial College London\2025 engineering\GNN molecules\graph_pickles\dataset02.xlsx"
data = pd.read_excel(env_file, engine='openpyxl').dropna(subset=['degradation_rate'])
data['seawater'] = data['seawater'].map({'art': 1, 'sea': 0})

folder_path = r"C:\Users\80710\OneDrive - Imperial College London\2025 engineering\GNN molecules\graph_pickles\molecules"
graph_pickles = [f for f in os.listdir(folder_path) if f.endswith(".pkl")]

base_dir = r"C:\Users\80710\OneDrive - Imperial College London\2025 engineering\GNN molecules\graph_pickles\molecules"
if os.path.exists(base_dir):
    print("Directory exists:", base_dir)
    print("Files in directory:", os.listdir(base_dir))
else:
    print(f"Error: Directory {base_dir} does not exist!")

compounds = data.component.unique()
graphs_dict = {}
for compound, graph_pickle in zip(compounds, graph_pickles):
    with open(os.path.join(base_dir, graph_pickle), 'rb') as f:
        graph = pickle.load(f)
        graphs_dict[compound] = networkx_to_pyg(graph)

dataset = MolDataset(
    raw_dataframe=data,
    nx_graph_dict=graphs_dict,
    component_col="component",
    global_state_cols=["temperature", "concentration", "time", "seawater"],
    label_col="degradation_rate",
    transform=None
)

# =============================================================================
#                      CROSS-VALIDATION (FIXED MODEL)
# =============================================================================
from torch_geometric.data import DataLoader as PyGDataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
k = 5  # number of folds
from sklearn.model_selection import KFold
kf = KFold(n_splits=k, shuffle=True, random_state=42)

fold_results = []

for fold, (train_idx, val_idx) in enumerate(kf.split(dataset)):
    print(f"\n--- Fold {fold + 1} ---")

    train_subset = torch.utils.data.Subset(dataset, train_idx)
    val_subset = torch.utils.data.Subset(dataset, val_idx)

    train_loader = PyGDataLoader(train_subset, batch_size=16, shuffle=True)
    val_loader = PyGDataLoader(val_subset, batch_size=16, shuffle=False)

    model = GINE_Regression(
        node_in_dim=1,
        edge_in_dim=2,
        external_in_dim=4,
        hidden_dim=16,  # Example hidden_dim
        num_layers=5,
        dropout=0.1
    ).to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
    criterion = torch.nn.MSELoss()

    num_epochs = 10
    for epoch in range(1, num_epochs + 1):
        train_loss = train_one_epoch(model, train_loader, optimizer, criterion, device)
        val_loss = validate(model, val_loader, criterion, device)

        if epoch % 10 == 0:
            print(f"[Fold {fold + 1} Epoch {epoch}] Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")

    print(f"Evaluating fold {fold + 1} ...")
    r2, rmse = evaluate_model(model, val_loader, device)
    fold_results.append({"fold": fold + 1, "r2": r2, "rmse": rmse})

r2_scores = [res["r2"] for res in fold_results]
rmse_scores = [res["rmse"] for res in fold_results]

print("\n--- Cross-Validation Summary ---")
for res in fold_results:
    print(f"Fold {res['fold']}: R² = {res['r2']:.4f}, RMSE = {res['rmse']:.4f}")

print(f"\nAverage R²: {np.mean(r2_scores):.4f} ± {np.std(r2_scores):.4f}")
print(f"Average RMSE: {np.mean(rmse_scores):.4f} ± {np.std(rmse_scores):.4f}")


# =============================================================================
#             IMPROVED MODEL WITH TRAPEZOID DIMENSIONS & PROJECTIONS
# =============================================================================
import optuna
import optuna.visualization as vis

def build_trapezoid_dims(num_layers):
    """
    Build a list of hidden dimensions in a trapezoid manner:
    up to the first 4 layers: [128, 64, 32, 16]
    if num_layers > 4, extend with 16 for extra layers.
    """
    base_dims = [128, 64, 32, 16]
    if num_layers > 4:
        extra_layers = num_layers - 4
        return base_dims + [16] * extra_layers
    else:
        return base_dims[:num_layers]


class GINE_RegressionTrapezoid(nn.Module):
    """
    A GINEConv-based regression model that uses a list of hidden dimensions
    to build layers with decreasing size (trapezoid architecture),
    ensuring dimension consistency with projection layers between convs.
    """
    def __init__(self,
                 node_in_dim: int,
                 edge_in_dim: int,
                 external_in_dim: int,
                 hidden_dims: list,
                 dropout: float = 0.1):
        super().__init__()

        # For the first layer, encode edges to hidden_dims[0], and encode nodes as well
        self.initial_edge_encoder = nn.Linear(edge_in_dim, hidden_dims[0])
        self.initial_node_encoder = nn.Linear(node_in_dim, hidden_dims[0])

        # We'll build each GINEConv to transform dimension: hidden_dims[i] -> hidden_dims[i].
        # After each conv i, if i < len(hidden_dims)-1, we project to hidden_dims[i+1].
        self.convs = nn.ModuleList()
        self.bns = nn.ModuleList()
        self.projections = nn.ModuleList()  # for node features
        self.edge_projections = nn.ModuleList()  # for edge features

        for i in range(len(hidden_dims)):
            # GINEConv's internal MLP
            net = nn.Sequential(
                nn.Linear(hidden_dims[i], hidden_dims[i]),
                nn.ReLU(),
                nn.Linear(hidden_dims[i], hidden_dims[i])
            )
            conv = GINEConv(nn=net)
            self.convs.append(conv)
            self.bns.append(BatchNorm(hidden_dims[i]))

            # If there's a next layer, we need a projection from hidden_dims[i] -> hidden_dims[i+1]
            if i < len(hidden_dims) - 1:
                self.projections.append(nn.Linear(hidden_dims[i], hidden_dims[i+1]))
                self.edge_projections.append(nn.Linear(hidden_dims[i], hidden_dims[i+1]))
            else:
                # No projection needed for the last layer
                self.projections.append(None)
                self.edge_projections.append(None)

        self.dropout_layer = nn.Dropout(p=dropout)

        # We'll process external factors to the final dimension
        final_dim = hidden_dims[-1]
        self.externals_mlp = nn.Sequential(
            nn.Linear(external_in_dim, final_dim),
            nn.ReLU(),
            nn.Dropout(p=dropout),
            nn.Linear(final_dim, final_dim)
        )

        # Final regression: combine final node features + final external features
        self.final_regressor = nn.Sequential(
            nn.Linear(final_dim + final_dim, final_dim),
            nn.ReLU(),
            nn.Dropout(p=dropout),
            nn.Linear(final_dim, 1)
        )

    def forward(self, data):
        x, edge_index, edge_attr, batch = data.x, data.edge_index, data.edge_attr, data.batch

        # 1) Encode node/edge to hidden_dims[0]
        x = self.initial_node_encoder(x)
        edge_emb = self.initial_edge_encoder(edge_attr)

        # 2) Pass through each GINEConv
        for i, (conv, bn) in enumerate(zip(self.convs, self.bns)):
            # GINEConv forward
            x = conv(x, edge_index, edge_emb)
            x = bn(x)
            x = F.relu(x)
            x = self.dropout_layer(x)

            # If there's a next layer, project node features and edge_emb to hidden_dims[i+1]
            if i < len(self.projections) - 1 and self.projections[i] is not None:
                x = self.projections[i](x)
                edge_emb = self.edge_projections[i](edge_emb)

        # 3) Global mean pooling
        graph_emb = global_mean_pool(x, batch)

        # 4) Process external factors into final dimension
        ext_emb = self.externals_mlp(data.externals)

        # 5) Concatenate and final regression
        combined = torch.cat([graph_emb, ext_emb], dim=-1)
        out = self.final_regressor(combined).squeeze(-1)
        return out


# =============================================================================
#                          OPTUNA OBJECTIVE FUNCTION
# =============================================================================

def objective(trial):
    """
    Objective function for Optuna.
    Performs k-fold cross validation using GINE_RegressionTrapezoid,
    returning negative average R². We log RMSE as a user attribute.
    """
    # Hyperparameter search space
    lr = trial.suggest_categorical("learning_rate", [1e-3, 3e-3, 1e-4, 3e-4])
    dropout = trial.suggest_categorical("dropout", [0.1, 0.5])
    num_layers = trial.suggest_int("num_layers", 2, 6)

    hidden_dims = build_trapezoid_dims(num_layers)
    num_epochs = 10  # Could also be a hyperparameter

    # Local 5-fold for CV
    kf_local = KFold(n_splits=5, shuffle=True, random_state=42)
    r2_scores = []
    rmse_scores = []

    for fold_idx, (train_idx, val_idx) in enumerate(kf_local.split(dataset)):
        train_subset = torch.utils.data.Subset(dataset, train_idx)
        val_subset = torch.utils.data.Subset(dataset, val_idx)

        train_loader = PyGDataLoader(train_subset, batch_size=16, shuffle=True)
        val_loader = PyGDataLoader(val_subset, batch_size=16, shuffle=False)

        model = GINE_RegressionTrapezoid(
            node_in_dim=1,
            edge_in_dim=2,
            external_in_dim=4,
            hidden_dims=hidden_dims,
            dropout=dropout
        ).to(device)

        optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)
        criterion = nn.MSELoss()

        for epoch in range(1, num_epochs + 1):
            train_loss = train_one_epoch(model, train_loader, optimizer, criterion, device)
            val_loss = validate(model, val_loader, criterion, device)

            # Report intermediate values (optional)
            if epoch % 10 == 0:
                trial.report(val_loss, step=epoch)
                if trial.should_prune():
                    raise optuna.TrialPruned()

        # Evaluate R² and RMSE for this fold
        model.eval()
        y_true_fold, y_pred_fold = [], []
        with torch.no_grad():
            for batch_data in val_loader:
                batch_data = batch_data.to(device)
                preds = model(batch_data)
                y_true_fold.append(batch_data.y.cpu())
                y_pred_fold.append(preds.cpu())

        y_true_fold = torch.cat(y_true_fold).numpy().squeeze()
        y_pred_fold = torch.cat(y_pred_fold).numpy().squeeze()

        fold_r2 = r2_score(y_true_fold, y_pred_fold)
        fold_rmse = np.sqrt(mean_squared_error(y_true_fold, y_pred_fold))

        r2_scores.append(fold_r2)
        rmse_scores.append(fold_rmse)

    avg_r2 = np.mean(r2_scores)
    avg_rmse = np.mean(rmse_scores)

    # Log RMSE as user attribute
    trial.set_user_attr("avg_rmse", float(avg_rmse))

    # Return negative R² because Optuna minimizes the objective
    return -avg_r2


# =============================================================================
#                           OPTUNA STUDY & DASHBOARD
# =============================================================================
if __name__ == "__main__":
    study = optuna.create_study(
        storage="sqlite:///gnn_mix_op05.sqlite3",
        study_name="GNN-mixed model",
        direction="minimize",
        load_if_exists=True
    )

    study.optimize(objective, n_trials=30, show_progress_bar=True)

    print("\n================= Optuna Study Results =================")
    best_trial = study.best_trial
    print(f"Best Trial Value (Negative R²): {best_trial.value}")
    print("Best Hyperparameters:")
    for key, val in best_trial.params.items():
        print(f"  {key}: {val}")
    print(f"User Attrs (e.g., RMSE): {best_trial.user_attrs}")

    try:
        fig1 = vis.plot_optimization_history(study)
        fig1.show()
    except Exception as e:
        print(f"Could not generate optimization history plot: {e}")

    try:
        fig2 = vis.plot_param_importances(study)
        fig2.show()
    except Exception as e:
        print(f"Could not generate hyperparameter importance plot: {e}")

    try:
        fig3 = vis.plot_intermediate_values(study)
        fig3.show()
    except Exception as e:
        print(f"Could not generate intermediate values plot: {e}")

    print("\n================= End of Optuna Tuning =================")


Directory exists: C:\Users\80710\OneDrive - Imperial College London\2025 engineering\GNN molecules\graph_pickles\molecules
Files in directory: ['gpickle_graph_0.pkl', 'gpickle_graph_1.pkl', 'gpickle_graph_10.pkl', 'gpickle_graph_11.pkl', 'gpickle_graph_12.pkl', 'gpickle_graph_13.pkl', 'gpickle_graph_14.pkl', 'gpickle_graph_15.pkl', 'gpickle_graph_16.pkl', 'gpickle_graph_17.pkl', 'gpickle_graph_18.pkl', 'gpickle_graph_19.pkl', 'gpickle_graph_2.pkl', 'gpickle_graph_3.pkl', 'gpickle_graph_4.pkl', 'gpickle_graph_5.pkl', 'gpickle_graph_6.pkl', 'gpickle_graph_7.pkl', 'gpickle_graph_8.pkl', 'gpickle_graph_9.pkl']

--- Fold 1 ---
[Fold 1 Epoch 10] Train Loss: 0.1751 | Val Loss: 0.0972
Evaluating fold 1 ...
R²: -0.7291
RMSE: 0.3117

--- Fold 2 ---
[Fold 2 Epoch 10] Train Loss: 0.1456 | Val Loss: 0.1514
Evaluating fold 2 ...
R²: -0.4495
RMSE: 0.3891

--- Fold 3 ---
[Fold 3 Epoch 10] Train Loss: 0.1790 | Val Loss: 0.0765
Evaluating fold 3 ...
R²: -0.0965
RMSE: 0.2767

--- Fold 4 ---
[Fold 4 Epoch

[I 2025-03-06 18:06:22,878] Using an existing study with name 'GNN-mixed model' instead of creating a new one.
Best trial: 1. Best value: 8.25468:   3%|▎         | 1/30 [01:35<46:05, 95.37s/it]

[I 2025-03-06 18:07:58,246] Trial 1 finished with value: 8.254682528706155 and parameters: {'learning_rate': 0.0003, 'dropout': 0.5, 'num_layers': 6}. Best is trial 1 with value: 8.254682528706155.


Best trial: 2. Best value: 6.04862:   7%|▋         | 2/30 [02:53<39:38, 84.95s/it]

[I 2025-03-06 18:09:15,895] Trial 2 finished with value: 6.0486228104918265 and parameters: {'learning_rate': 0.0003, 'dropout': 0.5, 'num_layers': 3}. Best is trial 2 with value: 6.0486228104918265.


...

[I 2025-03-06 18:42:22,053] Trial 28 finished with value: 0.02271963241202597 and parameters: {'learning_rate': 0.001, 'dropout': 0.1, 'num_layers': 3}. Best is trial 21 with value: -0.010473899258953302.


Best trial: 21. Best value: -0.0104739:  97%|█████████▋| 29/30 [36:16<00:51, 51.77s/it]

[I 2025-03-06 18:42:39,469] Trial 29 pruned. 


Best trial: 21. Best value: -0.0104739: 100%|██████████| 30/30 [36:35<00:00, 73.18s/it]

[I 2025-03-06 18:42:58,365] Trial 30 pruned. 

================= Optuna Study Results =================
Best Trial Value (Negative R²): -0.010473899258953302
Best Hyperparameters:
  learning_rate: 0.003
  dropout: 0.1
  num_layers: 3
User Attrs (e.g., RMSE): {'avg_rmse': 0.2757047116756439}
Could not generate optimization history plot: Tried to import 'plotly' but failed. Please make sure that the package is installed correctly to use this feature. Actual error: No module named 'plotly'.
Could not generate hyperparameter importance plot: Tried to import 'plotly' but failed. Please make sure that the package is installed correctly to use this feature. Actual error: No module named 'plotly'.
Could not generate intermediate values plot: Tried to import 'plotly' but failed. Please make sure that the package is installed correctly to use this feature. Actual error: No module named 'plotly'.

================= End of Optuna Tuning =================


---
---

###     Cross Validation

In [20]:
from sklearn.model_selection import KFold  #  k 折交叉验证

# ---------------------
# k-Fold Cross Validation
# ---------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
k = 5  # number of folds
kf = KFold(n_splits=k, shuffle=True, random_state=42) # shuffle = True 参数 随机打乱

fold_results = []

for fold, (train_idx, val_idx) in enumerate(kf.split(dataset)): # 循环 kf.split的折叠 enumerate函数添加计数
                                                                # fold 计数器 迭代次数
    print(f"\n--- Fold {fold + 1} ---")
    
    # Create train and validation subsets
    train_subset = torch.utils.data.Subset(dataset, train_idx) # 子集：训练集
    val_subset = torch.utils.data.Subset(dataset, val_idx)
    
    train_loader = PyGDataLoader(train_subset, batch_size=16, shuffle=True) # PyG 数据加载
    val_loader = PyGDataLoader(val_subset, batch_size=16, shuffle=False)
    
    # Initialize a new instance of the model, optimizer, and loss function for each fold
    model = GINE_Regression(
        node_in_dim=1,
        edge_in_dim=2,
        external_in_dim=4,
        hidden_dim=16,
        num_layers=5,
        dropout=0.1
    ).to(device)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
    criterion = torch.nn.MSELoss()
    
    num_epochs = 500
    for epoch in range(1, num_epochs + 1):
        train_loss = train_one_epoch(model, train_loader, optimizer, criterion, device)
        val_loss = validate(model, val_loader, criterion, device)
        
        if epoch % 10 == 0:
            print(f"[Fold {fold + 1} Epoch {epoch}] Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")
    
    print(f"Evaluating fold {fold + 1} ...")
    r2, rmse = evaluate_model(model, val_loader, device)
    fold_results.append({"fold": fold + 1, "r2": r2, "rmse": rmse})

# ---------------------
# Summary of Cross-Validation Results
# ---------------------
r2_scores = [res["r2"] for res in fold_results]
rmse_scores = [res["rmse"] for res in fold_results]

print("\n--- Cross-Validation Summary ---")
for res in fold_results:
    print(f"Fold {res['fold']}: R² = {res['r2']:.4f}, RMSE = {res['rmse']:.4f}")

print(f"\nAverage R²: {np.mean(r2_scores):.4f} ± {np.std(r2_scores):.4f}")
print(f"Average RMSE: {np.mean(rmse_scores):.4f} ± {np.std(rmse_scores):.4f}")


--- Fold 1 ---
[Fold 1 Epoch 10] Train Loss: 0.1210 | Val Loss: 0.0644
[Fold 1 Epoch 20] Train Loss: 0.0977 | Val Loss: 0.0594
[Fold 1 Epoch 30] Train Loss: 0.0972 | Val Loss: 0.0560
[Fold 1 Epoch 40] Train Loss: 0.0762 | Val Loss: 0.0503
[Fold 1 Epoch 50] Train Loss: 0.0821 | Val Loss: 0.0633
[Fold 1 Epoch 60] Train Loss: 0.0713 | Val Loss: 0.0479
[Fold 1 Epoch 70] Train Loss: 0.0706 | Val Loss: 0.0467
[Fold 1 Epoch 80] Train Loss: 0.0635 | Val Loss: 0.0454
[Fold 1 Epoch 90] Train Loss: 0.0611 | Val Loss: 0.0453
[Fold 1 Epoch 100] Train Loss: 0.0603 | Val Loss: 0.0442
[Fold 1 Epoch 110] Train Loss: 0.0595 | Val Loss: 0.0426
[Fold 1 Epoch 120] Train Loss: 0.0574 | Val Loss: 0.0488
[Fold 1 Epoch 130] Train Loss: 0.0613 | Val Loss: 0.0419
[Fold 1 Epoch 140] Train Loss: 0.0658 | Val Loss: 0.0567
[Fold 1 Epoch 150] Train Loss: 0.0585 | Val Loss: 0.0405
[Fold 1 Epoch 160] Train Loss: 0.0558 | Val Loss: 0.0384
[Fold 1 Epoch 170] Train Loss: 0.0468 | Val Loss: 0.0374
[Fold 1 Epoch 180] Train